# Putting all of the Pieces Together

Now that we know how to evaluate semantic similarity using a database, we can proceed to an actual RAG implementation.

In [ ]:
! pip install chromadb
! pip install ollama
! pip install nltk

In [2]:
import requests
import chromadb
import numpy as np
import uuid
import time
import nltk

from nltk.tokenize import PunktSentenceTokenizer

In [3]:
# Download the punkt tokenizer models that will help us split our text into sentences.
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dmitrystrakovsky/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
with open('./heart_attack.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [11]:
# Break our story up into seperate sentances
tokenizer = PunktSentenceTokenizer()
sentences = tokenizer.tokenize(text)

print(f"Total sentences: {len(sentences)}")
print(sentences[33])

Total sentences: 108
Alice couldn't help but interject.


## Now that our sentences are split, we can generate embeddings for individual sentences

In [12]:
# Using Nomic model served locally via Ollama for embedding
# Ollama is a friend --> https://ollama.com/
def get_embeddings_from_ollama(text, model="nomic-embed-text"):
    url = "http://localhost:11434/api/embeddings"
    
    payload = {
        "model": model,
        "prompt": text
    }
    
    response = requests.post(url, json=payload)
    return np.array(response.json()["embedding"], dtype=np.float32)

In [13]:
embeddings = []
for sentence in sentences:
    embedding = get_embeddings_from_ollama(sentence)
    embeddings.append(embedding)

## And now, on to packing everything into a database